In [1]:
import random
import numpy as np
import networkx as nx
import pandas as pd
A = 1
B = 720  # 8:00 am to 8 :00pm
COUNT = 100  # total order number
resultList = random.sample(range(A, B+1), COUNT)
order = []

In [2]:
def mapping(length, width):
    """
    With designated length and width, this function generate a length x width grid. And define the middle point as
    the location of restaurant.
    :param length: The number of nodes vertically
    :param width: The number of nodes horizontally
    :return: A NetworkX 2D gird graph
    """
    G = nx.grid_2d_graph(length, width)
    for edge in list(G.edges):
        low = np.random.randint(1, 10)
        high = np.random.randint(low + 1, 30)
        likely = np.random.randint(low, high)
        G.edges[edge[0], edge[1]]['low']=low
        G.edges[edge[0], edge[1]]['high']=high
        G.edges[edge[0], edge[1]]['likely']=likely
    G.nodes[int(length / 2), int(width / 2)]['name'] = 'RESTAURANT'
    return G

In [3]:
def real_map(G):
    for edge in list(G.edges):
        low= G.edges[edge[0], edge[1]]['low']
        high= G.edges[edge[0], edge[1]]['high']
        likely=G.edges[edge[0], edge[1]]['likely']
        distribution=mod_pert_random(low, likely, high, confidence=2)
        number=np.random.randint(0,len(distribution))
        G.edges[edge[0], edge[1]]['time'] = distribution[number]
    return G

In [4]:
def mod_pert_random(low, likely, high, confidence=4, samples=10000):
    """Produce random numbers according to the 'Modified PERT'
    distribution.
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        Currently allows values 1-18
    :param samples: random number size
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    if confidence < 1 or confidence > 18:
        raise ValueError('confidence value must be in range 1-18.')

    mean = (low + confidence * likely + high) / (confidence + 2)

    a = (mean - low) / (high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)

    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [5]:
a=list(real_map(mapping(2,2)).nodes)[1]


a

(0, 1)

In [6]:
class Order:
    def __init__(self, time, size, preparation_time = 0,order_location = (0,1),delivery_time = 0, supposed_delivered_time=0,delivered_time = 0,Status = 0,mark = 0):
        self.time = time
        self.size = size
        self.preparation_time = preparation_time
        self.order_location = order_location
        self.mark = mark
        self.delivery_time = delivery_time
        self.supposed_delivered_time = supposed_delivered_time
        self.delivered_time = delivered_time
        self.mark = mark
       
    def set_preparation_time(self):
        #implement former preparation function
        #### 时间是不对的！！！！!
        self.preparation_time = 10
        if self.size == 1:
            pass
        elif self.size == 2:
            self.preparation_time *= 1.5
        else:
            self.preparation_time *= 2
    
    def set_order_location (self, length,width,i):
        
        order_location=list(real_map(mapping(2,2)).nodes)[i]
        
        self.order_location = order_location
    
    def set_delivery_time (self,restaurant_loc, mapgrid):
        
        delivery_time = nx.dijkstra_path_length(mapgrid, source=restaurant_loc, target=self.order_location, weight='time')
        
        self.delivery_time = delivery_time
    
    def set_supposed_delivered_time(self):
        
        supposed_delivered_time = self.time + 100
        
        self.supposed_delivered_time = supposed_delivered_time
        
        
    def set_delivered_time(self, delivered_time):
        
        self.delivered_time = delivered_time
        
    def set_status(self):
        
        if self.delivered_time < self.supposed_delivered_time:
            self.status = 'processing'
        else:
            self.status = 'cancelled'
    def set_mark(self,mark):
        
        self.mark = mark
            
            
       
        
        

        
        
        
    
        
        
       


In [7]:
number = sorted(resultList)


In [8]:
new_map = real_map(mapping(2,2))
map_size = len(list(real_map(mapping(2,2)).nodes))

# define the location of a resturant
restaurant_loc = (int(2/2), int(2/2))

for i in range (100):
    
    size = random.sample(range(1,4), 1)[0]
    #order_location_index  is used to select a random node from the real_map
    order_location_index = random.sample(range(map_size), 1)[0]
    
    order.append(Order(number[i],size))
    
    order[i].set_preparation_time()
    order[i].set_order_location(2,2,order_location_index)
    order[i].set_delivery_time(restaurant_loc,new_map)
    order[i].set_supposed_delivered_time()


order[11].delivery_time
order[11].supposed_delivered_time

194

In [9]:
# marked the status of each order
# initiate the return time of the delivery time
arrived_time_of_deliverman = 0 
for i in range (100):
    if arrived_time_of_deliverman < order[i].time + order[i].preparation_time:
        
        delivered_time = order[i].time + order[i].preparation_time + order[i].delivery_time
    else:
        delivered_time = arrived_time_of_deliverman + order[i].delivery_time
    order[i].set_delivered_time (delivered_time)
    order[i].set_status()
    a=order[0]
    
    if order[i].status =='processing':
         arrived_time_of_deliverman = delivered_time + order[i].delivery_time
    else:
        for k in range (i)[::-1]:
            if order[k].status =='processing':
                arrived_time_of_deliverman = order[k].delivered_time + order[k].delivery_time
                break
       
            
    


In [10]:
status_list=[]
time_list=[]
delivered_list=[]
supposed_list=[]
for i in range(100):
    status_list.append(order[i].status)
    delivered_list.append(order[i].delivered_time)
    supposed_list.append(order[i].supposed_delivered_time)
    time_list.append(order[i].time)
    
status_list 
delivered_list
supposed_list
result = pd.DataFrame({'time': time_list, 'delivered': delivered_list,'supposed':supposed_list,'status':status_list})
result

,time,delivered,supposed,status
0,5,32.727416,105,processing
1,20,50.454832,120,processing
2,23,50.454832,123,processing
3,29,54.509704,129,processing
4,31,80.528094,131,processing
5,37,106.546485,137,processing
6,43,114.656228,143,processing
7,61,122.765971,161,processing
8,75,126.820843,175,processing
9,78,148.784362,178,processing


In [11]:
class DeliveryMan:
    def __init__(self, job_number, arrived_time = 0):
        self.job_number = job_number
        self.arrived_time = arrived_time
        
        
    def set_arrived_time (self, delivered_time, delivery_time):
        self.arrived_time = delivered_time + delivery_time
        
   
    
    


In [12]:
# arrived_time_of_deliverman = [0,0]
delivery_team = []
#(2)stands for the number of delivery man
# create objects 
for i in range (2):
    delivery_team.append(DeliveryMan(i+1))


for i in range (100):
    if i <= 1:
        delivered_time = order[i].time + order[i].preparation_time + order[i].delivery_time + 0
        order[i].set_delivered_time (delivered_time) 
        delivery_team[i].set_arrived_time(order[i].delivered_time , order[i].delivery_time )
        order[i].set_mark(i + 1)
    if i >= 2:
        if delivery_team[0].arrived_time >= delivery_team[1].arrived_time:
            mark = 1
            
        else:
            mark = 0
        arrived_time_of_deliverman = delivery_team[mark].arrived_time
        if arrived_time_of_deliverman < order[i].time + order[i].preparation_time:
        
            delivered_time = order[i].time + order[i].preparation_time + order[i].delivery_time
        else:
            delivered_time = arrived_time_of_deliverman + order[i].delivery_time
        
        order[i].set_status()
        order[i].set_delivered_time (delivered_time)
        
        if order[i].status == 'processing':
            arrived_time_of_deliverman = delivered_time + order[i].delivery_time
            order[i].set_mark(mark + 1)   
            
        else:
            for k in range (i)[::-1]:
                if order[k].status =='processing':
                    arrived_time_of_deliverman = order[k].delivered_time + order[k].delivery_time
                    break
        delivery_team[mark].arrived_time = arrived_time_of_deliverman
        
        
        



In [13]:
status_list = []
time_list = []
delivered_list = []
supposed_list = []
mark_list = []
for i in range(100):
    status_list.append(order[i].status)
    delivered_list.append(order[i].delivered_time)
    supposed_list.append(order[i].supposed_delivered_time)
    time_list.append(order[i].time)
    mark_list.append(order[i].mark)
    
    
    
result = pd.DataFrame({'time': time_list, 'delivered': delivered_list,'supposed':supposed_list,'status':status_list,'delivery_man':mark_list})
result

,time,delivered,supposed,status,delivery_man
0,5,32.727416,105,processing,1
1,20,40.000000,120,processing,2
2,23,40.000000,123,processing,2
3,29,44.054872,129,processing,2
4,31,70.073262,131,processing,2
5,37,56.054872,137,processing,1
6,43,67.054872,143,processing,1
7,61,75.164615,161,processing,1
8,75,90.000000,175,processing,1
9,78,119.963519,178,processing,1


In [14]:
# arrived_time_of_deliverman = [0,0]
k = 3
delivery_team = []
delivery_man_timelist = []
#(2)stands for the number of delivery man
# create objects 
for i in range (k):
    delivery_team.append(DeliveryMan(i+1))


for i in range (100):
    if i <= k-1:
        delivered_time = order[i].time + order[i].preparation_time + order[i].delivery_time + 0
        order[i].set_delivered_time (delivered_time) 
        delivery_team[i].set_arrived_time(order[i].delivered_time , order[i].delivery_time )
        order[i].set_mark(i + 1)
        delivery_man_timelist.append(delivery_team[i].arrived_time)
    if i >= k:
        mark = delivery_man_timelist.index(min(delivery_man_timelist))
#         print(mark)
#         if delivery_team[0].arrived_time >= delivery_team[1].arrived_time:
#             mark = 1
            
#         else:
#             mark = 0
        arrived_time_of_deliverman = delivery_team[mark].arrived_time
        if arrived_time_of_deliverman < order[i].time + order[i].preparation_time:
        
            delivered_time = order[i].time + order[i].preparation_time + order[i].delivery_time
        else:
            delivered_time = arrived_time_of_deliverman + order[i].delivery_time
        
        order[i].set_status()
        order[i].set_delivered_time (delivered_time)
        
        if order[i].status == 'processing':
            arrived_time_of_deliverman = delivered_time + order[i].delivery_time
            order[i].set_mark(mark + 1)   
            
        else:
            for k in range (i)[::-1]:
                if order[k].status =='processing':
                    arrived_time_of_deliverman = order[k].delivered_time + order[k].delivery_time
                    break
        delivery_team[mark].arrived_time = arrived_time_of_deliverman
        delivery_man_timelist[mark] = delivery_team[mark].arrived_time

        
   


In [15]:
status_list = []
time_list = []
delivered_list = []
supposed_list = []
mark_list = []
for i in range(100):
    status_list.append(order[i].status)
    delivered_list.append(order[i].delivered_time)
    supposed_list.append(order[i].supposed_delivered_time)
    time_list.append(order[i].time)
    mark_list.append(order[i].mark)
    
    
    
result = pd.DataFrame({'time': time_list, 'delivered': delivered_list,'supposed':supposed_list,'status':status_list,'delivery_man':mark_list})
result

,time,delivered,supposed,status,delivery_man
0,5,32.727416,105,processing,1
1,20,40.000000,120,processing,2
2,23,38.000000,123,processing,3
3,29,43.054872,129,processing,3
4,31,67.963519,131,processing,2
5,37,56.054872,137,processing,3
6,43,67.054872,143,processing,1
7,61,75.054872,161,processing,3
8,75,90.000000,175,processing,1
9,78,119.963519,178,processing,3
